<a href="https://colab.research.google.com/github/Dharshan4038/PDF_Q-A_USING_RAG/blob/main/RAG_APP_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# %pip install langchain ipykernel langchain_community pypdf langchain-text-splitters langchain-openai openai easyocr fitz frontend pymupdf faiss-cpu

In [43]:
# !pip install PyMuPDF easyocr pillow langchain

In [44]:
# !pip uninstall -y PyMuPDF
# !pip install PyMuPDF

In [45]:
# !pip list

In [46]:
!pip install pdf2image

In [47]:
import fitz  # PyMuPDF
import easyocr
import io
from PIL import Image
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings  # Updated for embeddings
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_openai import AzureOpenAIEmbeddings

In [48]:
# Extract images from PDF
def extract_image_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    image_paths = []

    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        images = page.get_images(full=True)

        for image_num, image in enumerate(images):
            xref = image[0]
            base_image = pdf_document.extract_image(xref)
            img_bytes = base_image["image"]
            img_pil = Image.open(io.BytesIO(img_bytes))
            img_path = f"/content/drive/MyDrive/output/image_{page_num}_{image_num}.png"
            img_pil.save(img_path)
            image_paths.append(img_path)

    return image_paths

In [49]:
# Extract text from images using EasyOCR
def extract_text_from_images_easyocr(image_paths):
    reader = easyocr.Reader(['en'])
    text = ""

    for image_path in image_paths:
        results = reader.readtext(image_path)
        for result in results:
            text += result[1] + "\n"

    return text

In [50]:
# Step 3: Preprocess the text and create a Document
def preprocess_text(text):
    lines = text.split("\n")
    for line in lines:
        print(line)

In [51]:
# Load text into Langchain's Document structure
def load_image_text_as_document(text):
    return Document(page_content=text, metadata={"source": "extracted_images"})

In [52]:
# Split documents into chunks
def split_docs(data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    return text_splitter.split_documents(data)

In [53]:
# Main flow for processing PDF and images
def process_pdf_and_images(pdf_path):
    # Extract images and text from the PDF
    image_paths = extract_image_from_pdf(pdf_path)
    text_from_images = extract_text_from_images_easyocr(image_paths)

    # Preprocess and create document for image-based text
    preprocess_text(text_from_images)
    img_doc = load_image_text_as_document(text_from_images)

    # Load the PDF content into Langchain's Document format
    pdf_loader = PyPDFLoader(pdf_path)
    pdf_docs = pdf_loader.load()

    # Append the image-based document to the PDF documents
    pdf_docs.append(img_doc)

    # Split all documents into smaller chunks
    split_documents = split_docs(pdf_docs)
    return split_documents

In [54]:
# Embedding and Vector Store creation
def create_faiss_vector_store(documents):
    # Initialize the embedding model
    embedding_model =  AzureOpenAIEmbeddings(
    azure_endpoint="https://llm-api-gateway-v2.azurewebsites.net",
    api_key="m8CHxff-D8N3PMQQBINku7JWGRX5JNJUeIDBcym286I",
    model="text-embedding-ada-002",
    azure_deployment="aiml-interns-embedding-ada-002-model",
  )

    # Create a FAISS vector store from the processed documents
    vectorstore = FAISS.from_documents(documents, embedding_model)
    return vectorstore

In [55]:
# Full execution
pdf_path = "/content/drive/MyDrive/RAG/test_case.pdf"
documents = process_pdf_and_images(pdf_path)

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

ME
ME
Test Data
validity
Test Case]
Test Casc?
Systet
Jmntolmat
Generation
uder
Test Casc}
Mechanism
Test
Artifact
Test Case n
Test
Oracle
ME
ME
Need for Literatulre Review
Research Questions
Planning Phase
Review Process
Search Criteria
Review" Selection Criteria
Conducting Phase
Review Selection Procedure
Data Extraction
Results & Discussion
Reporting Phase
Conclusion
ME
ME
ME
IEEE
Sprilger
189
ACM
Fist
Second
Third
Fiwal
Elsevier
rollnd
round
rolnd
Selection
Exclsion
Exchsion
Exchsion
Wiley
Google
Scholar
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME
ME



In [56]:
vectorstore = create_faiss_vector_store(documents)

In [57]:
from langchain_openai import AzureChatOpenAI
import os
os.environ["AZURE_OPENAI_API_KEY"] = "m8CHxff-D8N3PMQQBINku7JWGRX5JNJUeIDBcym286I"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://llm-api-gateway-v2.azurewebsites.net"

In [58]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [59]:
from langchain.vectorstores import  FAISS
from langchain.prompts import ChatPromptTemplate
template = """You are an AI assistant for question-answering tasks.
        Use the following pieces of context to answer the question. If you don't know the
        answer, please output 'No answer'.Use Maximum ten sentence and keep the answer concise
        Question: {question} Context: {context}
        """

In [60]:
prompt = ChatPromptTemplate.from_template(template)
from langchain.chat_models import AzureChatOpenAI
from langchain.schema.runnable import  RunnablePassthrough
from langchain.schema.output_parser import  StrOutputParser

In [61]:
output_parser = StrOutputParser()

In [62]:
llm = AzureChatOpenAI(
    azure_endpoint="https://llm-api-gateway-v2.azurewebsites.net",
    api_key="m8CHxff-D8N3PMQQBINku7JWGRX5JNJUeIDBcym286I",
    model = "gpt-35-turbo",
    max_tokens=50,
    azure_deployment="GPT-35-TURBO",
    api_version="2024-06-01"
)

In [63]:
retriever = vectorstore.as_retriever()

In [64]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
# Create a question-answering chain (QA Chain) that will use the LLM
qa_chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt)

# Define the ConversationalRetrievalChain to structure the pipeline
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [67]:
# Example question
question = "What is the percentage of publication in conference in public distribution ?"

# Initialize an empty chat history
chat_history = []

# Invoke the chain
try:
    response = chain({"question": question, "chat_history": chat_history})
    answer = response["answer"]
    source_documents = response["source_documents"]

    print("Answer:", answer)
    print("Source Documents:", source_documents)
except Exception as e:
    print(f"An error occurred: {e}")

Answer: The percentage of publication in conferences in the publication distribution is 21%.
Source Documents: [Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 8}, page_content='Figure 7. Publication distribution . \nPublication in \nConference\n21%\nPublication in \nJournal\n79%Publication Distribution\nPublication in Conference Publication in Journal'), Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 8}, page_content='2020  Mocha JS, \nNode JS  “Implementing DDD for Automatic Test Case Generation”  (Nachiengmai et al., \n2020)  \n \n \nAnswer to RQ 3 \nThe classification of research papers may help in tracking the research growth of the domain. The ratio \namongst the papers  published in conferences and journals of repute may give a clear idea about the \ngrowth. To provide the answer t o RQ 3, authors have classified the research papers in these two classes.  \n \nFigure 7 depict s that out of 1 25 selected papers, 7

In [68]:
question = "Name me the Input Language of testing tool used in test case genration in year of development/modification 2013 ?"

# Initialize an empty chat history
chat_history = []

# Invoke the chain
try:
    response = chain({"question": question, "chat_history": chat_history})
    answer = response["answer"]
    source_documents = response["source_documents"]

    print("Answer:", answer)
    print("Source Documents:", source_documents)
except Exception as e:
    print(f"An error occurred: {e}")

Answer: The input language of the testing tool used in test case generation in the year 2013 is C Language.
Source Documents: [Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 9}, page_content='Year of \nDevelopment/ \nModification  Name of Tool  Input Language  Type of Software  Reference  \n2010  Austin  C Language   Open Source  (Lakhotia et al., 2010)  \n2010  PET Java Academic & Research  (Albert et al., 2010)  \n2011  Korat  Java Open Source  (Boyapati et al., 2002)  \n2011  jPET  Java Commercial  (Albert et al., 2011)  \n2012  Palus  Java Open Source  (Zhang et al., 2011)  \n2013  MergePoint/ Mayhem  binary (32bit, Linux)  Commercial  (Avgerinos et al., 2014)  \nSoftware \nTesting\n74%Evolutionary \nTesting\n2%Regression \nTesting\n13%Others\n9%AI Testing\n2%RESEARCH DISTRIBUTION IN TESTING TYPE'), Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 10}, page_content='Verma et al.: Software Test Case Generation Tools a

In [69]:
question = "Name me the Title of the paper of dataset utilized in test case genration in year of publication 2015?"

# Initialize an empty chat history
chat_history = []

# Invoke the chain
try:
    response = chain({"question": question, "chat_history": chat_history})
    answer = response["answer"]
    source_documents = response["source_documents"]

    print("Answer:", answer)
    print("Source Documents:", source_documents)
except Exception as e:
    print(f"An error occurred: {e}")

Answer: The title of the paper utilizing the dataset "EvoSuite 10" in the year of publication 2015 is "A Memetic Algorithm for whole test suite generation" (Fraser et al., 2015).
Source Documents: [Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 12}, page_content='2016  Software \nTesting  Test Case \nGeneration  Toy Dataset  paper proposed a methodology \nfor generating test cases  Journal  (Sahoo et al., 2016b)'), Document(metadata={'source': '/content/drive/MyDrive/RAG/test_case.pdf', 'page': 12}, page_content='Generation  No Dataset  Survey/Review  Journal  (Pahwa and Solanki, \n2014)  \n2014  Software \nTesting  Test Suite \nGeneration  No Dataset  Survey/Review  Journal  (Hooda and Chhillar, \n2014)  \n2015  Evolutionary \nTesting  Test Suite \nGeneration  EvoSuite 10 tool  Memetic Algorithm for test \nsuite  optimization  Journal  (Fraser et al., 2015)  \n2016  Software \nTesting  Test Case \nGeneration  Toy Dataset  proposed a methodology for \n